In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
metal_files_list_f <- 'metal_input_pops.tsv'
phe_info_f <- '../../05_gbe/phenotype_info.tsv'
out_f <- 'icdinfo.metal.20200717.txt'


In [3]:
metal_files_list_f %>%
fread() %>%
rename('GBE_ID'='#GBE_ID') ->
metal_files_list_df


In [4]:
phe_info_f %>% fread() %>% 
rename('GBE_ID' = '#GBE_ID') -> phe_info_df


In [5]:
pop_str_df <- data.frame(
    pop_str = c('N_GBE', 'N_NBW', 'N_AFR', 'N_SAS', 'N_EAS', 'N_SMR', 'N_OTH'),
    pop = c('white_british','non_british_white','african','s_asian','e_asian','related','others')
)


In [6]:
pop_str_df

pop_str,pop
<fct>,<fct>
N_GBE,white_british
N_NBW,non_british_white
N_AFR,african
N_SAS,s_asian
N_EAS,e_asian
N_SMR,related
N_OTH,others


In [7]:
phe_info_df %>% 
select(
    GBE_ID, N_GBE, N_NBW, N_AFR, N_EAS, N_SAS, N_SMR, N_OTH
) %>%
gather(pop_str, N, -GBE_ID) %>%
left_join(pop_str_df, by='pop_str') %>%
select(-pop_str) %>%
inner_join(metal_files_list_df, c("GBE_ID", "pop")) %>%
group_by(GBE_ID) %>%
summarise(N_MA = sum(N)) %>%
ungroup() -> N_MA_df


`summarise()` ungrouping output (override with `.groups` argument)



In [8]:
phe_info_df %>% 
select(GBE_ID, GBE_NAME) %>% 
inner_join(N_MA_df, by = "GBE_ID") %>%
rename('N' = 'N_MA') -> icdinfo_df


In [9]:
icdinfo_df %>%
mutate(N2=N, N3=N, Y='Y') %>%
select(GBE_ID, N, GBE_NAME, N2, N3, Y) %>%
fwrite(out_f, sep='\t', na = "NA", quote=F, col.names=F)
